## Export packet loss for a selected perfsonar server

In [1]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

##  Select your link

In [2]:
site='UC'
packetLossServer = "192.170.227.160"

# site='IU'
# packetLossServer = "149.165.225.223"

# site='UIUC'
# packetLossServer = "72.36.96.4"

# site='ICCN'
# packetLossServer = "72.36.96.4"

## Get the data from Elasticsearch, and store the record based on its type

In [3]:
my_query = {
    "_source": ["dest","timestamp","packet_loss"],
    'query':{
            'bool':{
                   'must':[
                       {'range': {'timestamp': {'gte': '20180101T000000Z', 'lt': '20180401T000000Z'}}},
                       {'term': {'src': packetLossServer}}
                   ]
            }
    }
}

scroll = scan(client=es, index="ps_packet_loss", query=my_query)


In [10]:
epoch = datetime.utcfromtimestamp(0)
count = 0
dest=[]
pl=[]
ts=[]
for res in scroll:
    count += 1
    ts.append(res['_source']['timestamp']//1000)
    dest.append(res['_source']['dest'])
    pl.append(res['_source']['packet_loss'])
#     if count<2: print(res, ts)
    if not count%100000: print(count)
print(count)

data = pd.DataFrame({'dest':dest,'packet_loss':pl,'timestamp':ts})

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1073730


### saving in cvs files

In [15]:
data=data.set_index("timestamp")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1073730 entries, 1514764823 to 1522540798
Data columns (total 2 columns):
dest           1073730 non-null object
packet_loss    1073730 non-null float64
dtypes: float64(1), object(1)
memory usage: 24.6+ MB


In [17]:
data.sort_values(by='timestamp',inplace=True)    # Sort the records based on time
data.to_csv("Data/PacketLoss" + '_' + packetLossServer +".csv")

In [28]:
grouped_per_dest = data.groupby("dest")
for name, group in grouped_per_dest:
    group.drop("dest",axis=1,inplace=True)
    print(name)
    # deduplicate
    group = group[~group.index.duplicated(keep='first')]
    group.to_csv("Data/PacketLoss" + '_' + packetLossServer + '_to_' + name + ".csv",  index = False)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


109.105.124.86
117.103.105.191
128.142.223.247
129.107.255.26
129.107.255.29
129.15.40.231
129.215.213.70
129.93.5.165
130.209.239.124
130.246.176.109
130.246.47.129
131.111.66.196
131.169.98.30
132.206.245.252
134.158.123.183
134.158.132.200
134.158.159.85
134.158.20.192
134.158.73.243
134.158.84.141
134.219.225.13
134.79.118.72
138.253.60.82
141.108.35.18
141.34.200.28
142.150.19.61
144.206.236.189
144.206.237.142
145.100.17.8
146.83.90.7
147.156.116.40
147.213.204.112
147.231.25.192
148.187.64.25
149.165.224.247
149.165.225.1
149.165.225.223
149.165.225.3
150.244.246.85
157.82.112.68
158.195.14.26
163.1.5.210
164.58.29.121
192.101.107.152
192.108.47.12
192.114.101.125
192.12.15.111
192.12.15.26
192.170.227.163
192.170.230.51
192.231.127.41
192.41.230.61
192.41.236.31
192.5.207.251
192.54.207.250
193.109.172.188
193.170.243.215
193.48.99.76
194.190.165.192
194.36.11.38
194.80.35.169
194.85.69.75
195.194.105.178
200.17.30.135
202.122.32.170
206.12.127.30
206.12.154.60
206.12.9.2
72.36